In [2]:
!pip install selenium
!pip install baidu_aip
!pip install pillow

In [3]:
from selenium import webdriver
import matplotlib.pyplot as plt
import cv2
from PIL import Image
from aip import AipImageClassify, AipOcr
import re

from requests_html import HTMLSession
import requests
import time

session = HTMLSession()

# processOn网站 登录，自动连续下载测试

In [35]:
"""连接网站"""
def open_url(url, silence=False):
    r = session.get(url)                                                                #连接网站
    """设置下载路径"""
    options = webdriver.ChromeOptions() 
    direct = {'download.default_directory': 'C:\\Users\zelan\Desktop\网页数据抓取\下载测试'}     #更改下载默认地址为XXXX
    options.add_experimental_option('prefs', direct)
    if silence:
        options .add_argument('-headless')
    """打开浏览器"""
    browser = webdriver.Chrome(options=options)                                        #Chrome驱动
    browser.get(url)                                                                   #打开网页
    return browser

"""自动输入并登录"""
def auto_login(browser, id_user_name, id_password, user_name, password)
    browser.find_element_by_id(id_user_name).clear()                          #清空输入框
    browser.find_element_by_id(id_password).clear()                           #清空输入框
    browser.find_element_by_id(id_user_name).send_keys(user_name)             # 找到账号框并输入账号
    browser.find_element_by_id(id_password).send_keys(password)               # 找到密码框并输入密码
    browser.find_element_by_id('signin_btn').click()                          # 找到登陆按钮并点击
    return browser



"""在登陆后的界面干点啥"""
def open_project(browser, xpath):
    
    
    
    """打开标签页"""
    browser.find_element_by_xpath(xpath).click()
    """定位新标签页"""
    handles = browser.window_handles
    browser.switch_to.window(handles[-1])
    return browser

"""下载为PDF"""
def download_pdf(browser):
    browser.find_element_by_xpath('//*[@id="menu_bar_file"]').click()          #点击文件菜单
    #time.sleep(0.5)
    browser.find_element_by_xpath('//*[@id="bar_list_file"]/li[4]').click()    #点击下载
    #time.sleep(0.5)
    browser.find_element_by_xpath('//*[@id="export_pdf"]').click()             #点击pdf
    #time.sleep(0.5)
    browser.find_element_by_xpath('//*[@id="export_ok"]').click()              #点击确定
    return browser

def process_on_main():
    url = 'https://www.processon.com/login'
    id_user_name = 'login_email'
    id_password = 'login_password'
    user_name = '****'
    password = '****'

    browser = open_url(url)
    time.sleep(1)
    browser = auto_login(browser, id_user_name, id_password, user_name, password)
    time.sleep(1)
    
    xpath = '/html/body/div[2]/div[2]/div[%d]'
    """定位当前页（列表页）"""
    windows = browser.current_window_handle
    project_num = 2
    while 1:
        project_num+=1
        try:
            browser.switch_to.window(windows)
            browser = open_project(browser, xpath%project_num)
            time.sleep(1)
            browser = download_pdf(browser)
            time.sleep(1)
            browser.close()
            
        except:
            break
    browser.switch_to.window(windows)
    return browser

def logout(browser):
    browser.find_element_by_xpath('//*[@id="user-logo"]').click()
    browser.find_element_by_xpath('/html/body/ul/li[10]/a').click()

    return browser

In [38]:
"""打开，登录，下载"""
session = HTMLSession()
browser = process_on_main()
#browser = logout(browser)

In [10]:
"""关闭浏览器"""
browser.quit()

# 起点中文网 批量爬取文字测试

In [5]:
#from requests_html import HTMLSession
from selenium import webdriver
from time import sleep
from string import digits
from os import path, makedirs,getcwd
from selenium.common.exceptions import NoSuchElementException
from tkinter import *
from tkinter import ttk, filedialog
#session = HTMLSession()

In [6]:
from selenium.webdriver.chrome.options import Options

chrome_options = Options()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--headless')
browser = webdriver.Chrome(chrome_options=chrome_options)

D:\anaconda\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: use options instead of chrome_options
  import sys


In [34]:
"""判断/建立文件夹"""
def get_folder(book_path):
    exist = path.exists(book_path)
    if not exist:
        makedirs(book_path)
    return book_path

"""判断文件是否存在"""
def check_txt(text_path):
    exist = path.exists(text_path)
    return exist


"""定位新标签页"""
def loc_page(browser):
    handles = browser.window_handles
    browser.switch_to.window(handles[-1])
    return browser

"""获取内容"""
def get_content(browser):
    content = browser.find_element_by_class_name('j_readContent').text
    charpter_name = browser.find_element_by_class_name('j_chapterName').text.rstrip(digits).replace('\n','')
    chapter_content = charpter_name + '\n' + content
    return charpter_name, chapter_content

def open_book(browser, xpath, number):
    browser.find_element_by_xpath(xpath).click()
    """定位新标签页"""
    browser = loc_page(browser)
    book_name = browser.find_element_by_xpath('/html/body/div/div[6]/div[1]/div[2]/h1/em').text
    """打开目录"""
    menu_xpath = '//*[@id="j_catalogPage"]'
    browser.find_element_by_xpath(menu_xpath).click()
    """打开第一章"""
    browser.find_element_by_xpath('//*[@id="j-catalogWrap"]/div[2]/div/ul/li[1]/a').click()
    """定位新标签页"""
    browser = loc_page(browser)
    """尝试关闭弹出的窗口"""
    try:    
        browser.find_element_by_xpath('//*[@id="j_closeGuide"]').click()
    except NoSuchElementException:
        
        pass
    
    folder_path = get_folder('小说/'+book_name)
    while 1:
        if number == 0:
            break
        number -= 1
        """获取本章内容"""
        try:
            charpter_name, charpter_content = get_content(browser)
            
        except NoSuchElementException:
            
            break
        """存入书目文件夹"""
        
        charpter_path = folder_path + '/%s.txt'%(charpter_name)
        with open(charpter_path,"w") as f:
            charpter.write(charpter_content)        
            charpter.close()
        
        #print(book_name +' '+ charpter_name)
        """跳转下一页"""
        browser.find_element_by_id('j_chapterNext').click()
    """关闭章节标签页"""
    browser.close()
    
    return loc_page(browser)

"""打开第X本书的X页"""
def read_X_chapter(browser, windows, book_num, chapt_num):
    """跳转列表页"""
    browser.switch_to.window(windows)
    """找到第一个作品xpath"""
    O_xpath = '//*[@id="free-channel-wrap"]/div/div/div[2]/div[2]/div/ul/li[%d]/div[1]/a'
    xpath = O_xpath%book_num
    """打开第一个作品"""
    browser = open_book(browser, xpath, chapt_num)
    browser.close()
    
"""打开免费作品列表"""
def open_free_page(browser):
    """打开免费小说"""
    browser.find_element_by_xpath('/html/body/div[1]/div[4]/div/ul/li[5]/a').click()

    """打开免费作品"""
    browser.find_element_by_xpath('//*[@id="free-type-tab"]/ul/li[2]').click()
    return browser
    
"""连接网站"""
def open_url(url, silence=False):
    #r = session.get(url)                                                                #连接网站
    """设置下载路径"""
    options = webdriver.ChromeOptions() 
    direct = {'download.default_directory': 'C:\\Users\zelan\Desktop\小说抓取\小说'}     #更改下载默认地址为XXXX
    options.add_experimental_option('prefs', direct)
    if silence:
        options .add_argument('-headless')
    """打开浏览器"""
    browser = webdriver.Chrome(options=options)                                        #Chrome驱动
    browser.get(url)                                                                   #打开网页
    return browser

In [35]:
"""进度条"""
def progress_bar(i, book_num):
    sleep(0.01)
    v = int(100/book_num)
    for k in range(v):
        sleep(0.1)
        process = "\r[%3s%%]: |%-50s" % (k+1+i*v, '█' * (int(k+1+i*v)//2))
        if i == (book_num-1) and k == v-1:
            process = "\r[Done]: |%-50s" % ('█' * 50)
        print(process, end='', flush=True)    
    
"""打开起点"""
def get_novel_qidian(book_num, charpter_num):
    url = 'https://www.qidian.com'
    browser = open_url(url, silence=False)
    """打开免费作品列表"""
    browser = open_free_page(browser)
    windows = browser.current_window_handle
    """开始下载"""
    
    for i in range(1,book_num+1):
        read_X_chapter(browser, windows, i, charpter_num)
        """进度条"""
        progress_bar(i, book_num)

    """关闭浏览器"""
    browser.quit()

 

In [36]:
book_num = 10
charpter_num = 1 #所有章节
get_novel_qidian(int(book_num), charpter_num)

[ 90%]: |█████████████████████████████████████████████     

WebDriverException: Message: chrome not reachable
  (Session info: chrome=87.0.4280.141)


## UI

In [11]:
def search_book(book_name, book_num):
    browser.find_element_by_xpath('//*[@id="s-box"]').send_keys(book_name)
    browser.find_element_by_xpath('//*[@id="search-btn"]').click()
    return loc_page(browser)


"""进度条"""
def UI_progress_bar(i, book_num):
    sleep(0.01)
    persent = (i+1) / book_num
    v = int(100/book_num)
    
    for k in range(v):
        VALUE3_BOX['value'] =  int(k+1+i*v)
        ROOT.update()    
        

        
"""UI打开起点"""
def UI_get_novel_qidian(book_num, charpter_num, book_name):
    url = 'https://www.qidian.com'
    browser = open_url(url, silence=False)
    
    if book_name != '':
        browser = search_book(book_name, book_num)
        book_num = 1
    
    else:
        """打开免费作品列表"""
        browser = open_free_page(browser)
    windows = browser.current_window_handle
    """开始下载"""
    for i in range(1,book_num+1):
        read_X_chapter(browser, windows, i, charpter_num)
        """进度条"""
        UI_progress_bar(i, book_num)

    """关闭浏览器"""
    browser.quit()   
    
        
"""开始"""
def get_start(choise, book_num, book_name):
    """检查按钮状态"""
    if INSERT_BUTTON['text']=='完成':
        VALUE4_BOX['state'] = 'normal'
        VALUE1_BOX['state'] = 'normal'
        VALUE2_BOX['state'] = 'normal'
        VALUE3_BOX['value'] =  0
        INSERT_BUTTON['text']='开始'
        ROOT.update()
    elif INSERT_BUTTON['text']=='读取中':
        pass
    else:
        """确定网站"""
        if choise == '起点中文网':
            """禁用输入框"""
            VALUE4_BOX['state'] = 'disabled'
            VALUE1_BOX['state'] = 'disabled'
            VALUE2_BOX['state'] = 'disabled'
            
            """开始读取并下载"""
            
            charpter_num = -1
            INSERT_BUTTON['text']='读取中'
            ROOT.update()
            UI_get_novel_qidian(int(book_num), charpter_num, book_name)
            INSERT_BUTTON['text']='完成'
            ROOT.update()
    

In [ ]:

#GUI
ROOT = Tk()
"""标题"""
ROOT.title('小说爬虫')
"""界面大小"""
ROOT.geometry()
"""分区"""
"""大区"""
SUPERFRAME = Frame(ROOT, bd=25) #大区
"""中区，上下分大区"""
BIGFRAME1 = Frame(SUPERFRAME, bd=10) #中区1
BIGFRAME2 = Frame(SUPERFRAME, bd=10) #中区2
"""小区，上下分中区"""
FRAME1 = Frame(BIGFRAME1, bd=10) #小区1
FRAME2 = Frame(BIGFRAME1, bd=20) #小区2

FRAME3 = Frame(BIGFRAME2, bd=10) #小区1
FRAME4 = Frame(BIGFRAME2, bd=10) #小区2

"""细区，分小区"""
MINI_FRAME1 = Frame(FRAME1, bd=10)
MINI_FRAME2 = Frame(FRAME1, bd=10)

"""选择网站"""
VALUE1_TEXT = Label(MINI_FRAME1, text='获取网站:')
DEFAULT_VALUE = StringVar()
VALUE1_BOX = ttk.Combobox(MINI_FRAME1,textvariable=DEFAULT_VALUE, width=27)
VALUE1_BOX['value'] = ('起点中文网', '其他')
VALUE1_BOX.current(0)

VALUE1_TEXT.pack(side=LEFT)
VALUE1_BOX.pack(side=RIGHT)


"""输入书名"""
VALUE4_TEXT = Label(MINI_FRAME2, text='搜索书名:')
VALUE4_BOX = Entry(MINI_FRAME2)

VALUE4_TEXT.pack(side=LEFT)
VALUE4_BOX.pack(side=RIGHT)



"""选择下载数量"""
VALUE2_TEXT = Label(FRAME2, text='获取本数:')
VALUE2_BOX = Spinbox(FRAME2,from_=0, to=1000, width=28)

VALUE2_TEXT.pack(side=LEFT)
VALUE2_BOX.pack(side=RIGHT)



"""进度显示"""
VALUE3_TEXT = Label(FRAME3, text='读取进度:', justify=LEFT)
VALUE3_BOX = ttk.Progressbar(FRAME3, length = 210, value = 0, mode = 'determinate')


VALUE3_TEXT.pack(side=LEFT)
VALUE3_BOX.pack(side=RIGHT)
VALUE3_BOX['maximum'] = 100
VALUE3_BOX['value'] = 0



INSERT_BUTTON = Button(FRAME4, text='开始', width=10, height=1,
                       font=('microsoft yahei',10, 'bold'),
                       command=lambda: get_start(VALUE1_BOX.get(), VALUE2_BOX.get(), VALUE4_BOX.get())
                      )
INSERT_BUTTON.pack()

MINI_FRAME1.pack(side=TOP, fill='both')
MINI_FRAME2.pack(side=BOTTOM, fill='both')
FRAME3.pack(side=TOP, fill='both')
FRAME4.pack(side=BOTTOM, fill='both')
FRAME1.pack(side=TOP, fill='both')
FRAME2.pack(side=BOTTOM, fill='both')
BIGFRAME1.pack(side=TOP)
BIGFRAME2.pack(side=BOTTOM)
SUPERFRAME.pack()



ROOT.mainloop()

In [55]:
url = 'https://www.qidian.com'
browser1 = webdriver.Chrome()                                        #Chrome驱动
browser1.get(url)


In [56]:
browser1.find_element_by_xpath('//*[@id="s-box"]').send_keys('name')
browser1.find_element_by_xpath('//*[@id="search-btn"]').click()

In [8]:

v = int(100)
for k in range(v):

    process = "\r[%3s%%]: |%-50s" % (k, '█' * (int(k+1)//2))
    
    print(process, end='', flush=True)   

[ 99%]: |██████████████████████████████████████████████████

In [9]:
2//2

1

In [10]:
3//2


1

In [11]:
5//2


2

In [50]:
from tkinter import *
from tkinter import ttk, filedialog
def video_path():
    global video_file_path
    f = filedialog.askopenfilename()

    VALUE_BOX.insert(0,f) 
    ROOT.update() 
    video_file_path = VALUE_BOX.get()
    
def image_path():
    global image_file_path
    f = filedialog.askopenfilename()

    VALUE_BOX2.insert(0,f) 
    ROOT.update() 
    image_file_path = VALUE_BOX2.get()

video_file_path = ''
image_file_path = ''
#GUI
ROOT = Tk()
"""标题"""
ROOT.title('小说爬虫')
"""界面大小"""
ROOT.geometry()
"""分区"""
"""大区"""
SUPERFRAME = Frame(ROOT, bd=25) #大区
SUPERFRAME2 = Frame(ROOT, bd=25) #大区
"""中区，上下分大区"""
BIGFRAME1 = Frame(SUPERFRAME, bd=10) #中区1
BIGFRAME2 = Frame(SUPERFRAME, bd=10) #中区2

VALUE_TEXT = Label(BIGFRAME1, text='video:')
VALUE_BOX = Entry(BIGFRAME1)

VALUE_TEXT.pack(side=LEFT)
VALUE_BOX.pack(side=LEFT)
INSERT_BUTTON = Button(BIGFRAME1, text='开始', width=10, height=1,
                       font=('microsoft yahei',10, 'bold'),
                       command=lambda: video_path()
                      )
INSERT_BUTTON.pack()

VALUE_TEXT2 = Label(BIGFRAME2, text='image:')
VALUE_BOX2 = Entry(BIGFRAME2)

VALUE_TEXT2.pack(side=LEFT)
VALUE_BOX2.pack(side=LEFT)
INSERT_BUTTON2 = Button(BIGFRAME2, text='开始', width=10, height=1,
                       font=('microsoft yahei',10, 'bold'),
                       command=lambda: image_path()
                      )
INSERT_BUTTON2.pack()




INSERT_BUTTON2 = Button(SUPERFRAME2, text='Close', width=10, height=1,
                       font=('microsoft yahei',10, 'bold'),
                       command=lambda: ROOT.destroy()
                      )
INSERT_BUTTON2.pack()
BIGFRAME1.pack(side=TOP)
BIGFRAME2.pack(side=BOTTOM)
SUPERFRAME.pack(side=TOP)
SUPERFRAME2.pack(side=BOTTOM)

ROOT.mainloop()

print (video_file_path)
print (image_file_path)

C:/Users/zelan/OneDrive/文档/vc_pic2.png
C:/Users/zelan/OneDrive/文档/vc_pic.png


In [1]:
!pip freeze > requirements.txt

In [ ]:
//*[@id="limit-list"]/div/ul/li[1]/div[3]/p/a[1]